# Лабораторная работа № 1
## Работа с алгоритмом KNN

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    classification_report
)
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error, 
    r2_score
)

## Загрузка данных

In [3]:
# ===== КЛАССИФИКАЦИЯ =====
clf_data = pd.read_csv("datasets/smoke_detection_iot.csv")
print(clf_data.head(5))

clf_X = clf_data.drop(columns=["Fire Alarm", "Unnamed: 0", "CNT"])
clf_y = clf_data["Fire Alarm"]

print("Classification dataset shape:", clf_data.shape)


# ===== РЕГРЕССИЯ =====
reg_data = pd.read_csv("datasets/used_cars.csv")
print(reg_data.head(5))

reg_y = reg_data["price"]
reg_X = reg_data.drop(columns=["price"])

# для категориальных признаков
reg_X = pd.get_dummies(reg_X, drop_first=True)

# Приведение цены к числовому типу
reg_y = (
    reg_data["price"]
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .astype(float)
)

print("Regression dataset shape:", reg_data.shape)

   Unnamed: 0         UTC  Temperature[C]  Humidity[%]  TVOC[ppb]  eCO2[ppm]  \
0           0  1654733331          20.000        57.36          0        400   
1           1  1654733332          20.015        56.67          0        400   
2           2  1654733333          20.029        55.96          0        400   
3           3  1654733334          20.044        55.28          0        400   
4           4  1654733335          20.059        54.69          0        400   

   Raw H2  Raw Ethanol  Pressure[hPa]  PM1.0  PM2.5  NC0.5  NC1.0  NC2.5  CNT  \
0   12306        18520        939.735    0.0    0.0    0.0    0.0    0.0    0   
1   12345        18651        939.744    0.0    0.0    0.0    0.0    0.0    1   
2   12374        18764        939.738    0.0    0.0    0.0    0.0    0.0    2   
3   12390        18849        939.736    0.0    0.0    0.0    0.0    0.0    3   
4   12403        18921        939.744    0.0    0.0    0.0    0.0    0.0    4   

   Fire Alarm  
0           0  


## Детальное изучение данных

Перед началом обработки давайте внимательно изучим структуру и особенности наших датасетов.

In [15]:
# ===== ИССЛЕДОВАНИЕ ДАННЫХ КЛАССИФИКАЦИИ (детекция дыма) =====
print("="*60)
print("ДАННЫЕ ДЛЯ КЛАССИФИКАЦИИ (Smoke Detection)")
print("="*60)

print(f"Размер датасета: {clf_data.shape}")
print(f"Столбцы: {list(clf_data.columns)}")

print("\n--- Информация о типах данных ---")
print(clf_data.dtypes)

print("\n--- Информация о пропусках ---")
print(clf_data.isnull().sum())

print("\n--- Целевая переменная (Fire Alarm) ---")
print("Распределение классов:")
print(clf_y.value_counts())
print("Доли классов:")
print(clf_y.value_counts(normalize=True))

print("\n--- Примеры записей ---")
print("Первые 3 записи:")
print(clf_data.head(3))
print("\nСлучайные 3 записи:")
print(clf_data.sample(3, random_state=42))

print("\n--- Базовая статистика признаков ---")
print(clf_X.describe())

ДАННЫЕ ДЛЯ КЛАССИФИКАЦИИ (Smoke Detection)
Размер датасета: (62630, 16)
Столбцы: ['Unnamed: 0', 'UTC', 'Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0', 'NC2.5', 'CNT', 'Fire Alarm']

--- Информация о типах данных ---
Unnamed: 0          int64
UTC                 int64
Temperature[C]    float64
Humidity[%]       float64
TVOC[ppb]           int64
eCO2[ppm]           int64
Raw H2              int64
Raw Ethanol         int64
Pressure[hPa]     float64
PM1.0             float64
PM2.5             float64
NC0.5             float64
NC1.0             float64
NC2.5             float64
CNT                 int64
Fire Alarm          int64
dtype: object

--- Информация о пропусках ---
Unnamed: 0        0
UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5        

In [ ]:
# ===== ИССЛЕДОВАНИЕ ДАННЫХ РЕГРЕССИИ (цены автомобилей) =====
print("\n" + "="*60)
print("ДАННЫЕ ДЛЯ РЕГРЕССИИ (Used Car Prices)")
print("="*60)

print(f"Размер датасета: {reg_data.shape}")
print(f"Столбцы: {list(reg_data.columns)}")

print("\n--- Информация о типах данных ---")
print(reg_data.dtypes)

print("\n--- Примеры записей (исходные данные) ---")
print(reg_data.head(3))

print("\n--- Детальный анализ каждого столбца ---")
for col in reg_data.columns:
    print(f"\n Столбец '{col}':")
    print(f"   Тип данных: {reg_data[col].dtype}")
    print(f"   Уникальных значений: {reg_data[col].nunique()}")
    print(f"   Пропуски: {reg_data[col].isnull().sum()}")
    
    if reg_data[col].dtype == 'object':
        print(f"   Примеры значений: {reg_data[col].unique()[:5]}")
        if reg_data[col].nunique() < 20:
            print(f"   Все уникальные значения: {reg_data[col].unique()}")
    else:
        print(f"   Мин: {reg_data[col].min()}, Макс: {reg_data[col].max()}")

print("\n--- Анализ целевой переменной (price) ---")
print("Примеры цен (исходный формат):")
print(reg_data['price'].head(10))

print("\nПримеры цен (после обработки):")
print(reg_y.head(10))
print(f"Статистика цен: min={reg_y.min():.2f}, max={reg_y.max():.2f}, mean={reg_y.mean():.2f}")


ДАННЫЕ ДЛЯ РЕГРЕССИИ (Used Car Prices)
Размер датасета: (4009, 12)
Столбцы: ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']

--- Информация о типах данных ---
brand           object
model           object
model_year       int64
milage          object
fuel_type       object
engine          object
transmission    object
ext_col         object
int_col         object
accident        object
clean_title     object
price           object
dtype: object

--- Примеры записей (исходные данные) ---
     brand                            model  model_year      milage  \
0     Ford  Utility Police Interceptor Base        2013  51,000 mi.   
1  Hyundai                     Palisade SEL        2021  34,742 mi.   
2    Lexus                    RX 350 RX 350        2022  22,372 mi.   

       fuel_type                                             engine  \
0  E85 Flex Fuel  300.0HP 3.7L V6 Cylinder Engine Flex Fuel

### Выводы по изучению данных

**Данные классификации (детекция дыма) — 62,630 записей, 15 признаков:**
 **Хорошее качество данных:**
- Нет пропусков
- Все признаки числовые
- Классы умеренно несбалансированы: 71% (огонь) vs 29% (нет огня)

 **Требует внимания:**
- Признаки имеют очень разные масштабы (UTC: ~1.6B, температура: ~20, частицы: 0-60k)
- Некоторые признаки могут иметь выбросы (max TVOC: 60,000 vs median: 981)

**Данные регрессии (цены автомобилей) — 4,009 записей, 11 признаков:**
 **Серьезные проблемы с данными:**
- Множество пропусков: fuel_type (170), accident (113), clean_title (596)
- Пробег в текстовом формате: "51,000 mi."
- Огромный диапазон цен: $2,000 - $2,954,083 (есть выбросы!)
- Очень много уникальных значений в категориях: model (1,898), engine (1,146)

 **Критические проблемы:**
- `clean_title` содержит только одно значение 'Yes' → бесполезный признак
- `fuel_type` содержит странные значения: '–', 'not supported'

**Обновленный план действий:**
1. **Классификация**: простая нормализация + проверка выбросов
2. **Регрессия**: серьезная очистка данных + обработка категориальных признаков + удаление выбросов в ценах

## Базовые бейзлайны

In [4]:
# ===== Классификация =====
clf_X_train, clf_X_test, clf_y_train, clf_y_test = train_test_split(
    clf_X, clf_y,
    test_size=0.2,
    random_state=42,
    stratify=clf_y
)

clf_scaler = StandardScaler()
clf_X_train_scaled = clf_scaler.fit_transform(clf_X_train)
clf_X_test_scaled = clf_scaler.transform(clf_X_test)

clf_knn = KNeighborsClassifier()
clf_knn.fit(clf_X_train_scaled, clf_y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [5]:
# ===== Регрессия =====
reg_X_train, reg_X_test, reg_y_train, reg_y_test = train_test_split(
    reg_X, reg_y,
    test_size=0.2,
    random_state=42
)

reg_scaler = StandardScaler()
reg_X_train_scaled = reg_scaler.fit_transform(reg_X_train)
reg_X_test_scaled = reg_scaler.transform(reg_X_test)

reg_knn = KNeighborsRegressor()
reg_knn.fit(reg_X_train_scaled, reg_y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [6]:
# ===== Предсказания =====
y_pred = clf_knn.predict(clf_X_test_scaled)
y_proba = clf_knn.predict_proba(clf_X_test_scaled)[:, 1]

# ===== Метрики =====
accuracy = accuracy_score(clf_y_test, y_pred)
precision = precision_score(clf_y_test, y_pred)
recall = recall_score(clf_y_test, y_pred)
f1 = f1_score(clf_y_test, y_pred)
roc_auc = roc_auc_score(clf_y_test, y_proba)

cm = confusion_matrix(clf_y_test, y_pred)

# ===== Вывод =====
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print("Confusion matrix:")
print(cm)

Accuracy:  0.9992
Precision: 0.9991
Recall:    0.9998
F1-score:  0.9994
ROC-AUC:   0.9997
Confusion matrix:
[[3567    8]
 [   2 8949]]


Метрики показывают почти идеальное качество модели. Это значит, что модель почти не делает ошибок при выявлении как наличия, так и отсутствия огня. Особенно важен высокий Recall, который говорит о том, что практически все настоящие случаи пожара выявляются — критически важная характеристика для системы раннего обнаружения. Низкое число ложных срабатываний подтверждается высоким Precision и симметричной матрицей ошибок, где практически все объекты классифицируются корректно. В целом, baseline KNN уже даёт очень хороший результат, и улучшения могут быть направлены только на стабилизацию модели и оптимизацию вычислений.

In [7]:
# ===== Предсказания =====
reg_y_pred = reg_knn.predict(reg_X_test_scaled)

# ===== Метрики =====
mae = mean_absolute_error(reg_y_test, reg_y_pred)
mse = mean_squared_error(reg_y_test, reg_y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(reg_y_test, reg_y_pred)

print(f"MAE:  {mae:.2f}")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2:   {r2:.4f}")

MAE:  30653.82
MSE:  20366344234.70
RMSE: 142710.70
R2:   0.0036


Метрики показывают крайне слабое качество предсказаний. Это означает, что модель практически не способна объяснять вариацию цены автомобилей и выдаёт предсказания, близкие к среднему значению по выборке, игнорируя индивидуальные характеристики. Очень высокий RMSE и почти нулевой R² показывают, что baseline KNN на исходных признаках без обработки категориальных переменных и нормализации сильно ограничен. Для улучшения результатов потребуется тщательный препроцессинг: преобразование строковых признаков (марка, модель, тип коробки), корректное масштабирование числовых переменных, создание новых признаков и подбор гиперпараметров модели.

## Улучшение бейзлайнов

### Формулировка гипотез для улучшения бейзлайна

**Классификация (детекция дыма/огня):**

Анализ данных показал высокое качество датасета: отсутствие пропусков, все признаки числовые. Однако выявлена основная проблема - значительные различия в масштабах признаков (UTC: ~1.6B, температура: ~20). Также наблюдается дисбаланс классов (71% vs 29%).

Гипотезы для улучшения:
1. Применение робастной нормализации признаков повысит качество модели
2. Подбор оптимальных гиперпараметров KNN (количество соседей, веса, метрика расстояния) улучшит performance
3. Балансировка классов методом SMOTE может повысить recall модели

**Регрессия (цены автомобилей):**

Данные требуют существенной предобработки. Выявлены критические проблемы: пробег в текстовом формате, экстремальные выбросы в ценах (до $2.9M), множественные пропуски в ключевых признаках.

Гипотезы для улучшения:
1. Очистка и преобразование признака пробега в числовой формат устранит ошибки обработки
2. Удаление выбросов в ценах повысит стабильность модели
3. Создание новых признаков (возраст автомобиля, группировка редких категорий) улучшит предсказательную способность
4. Извлечение структурированной информации из текстовых полей (engine, transmission) добавит ценные признаки
5. Применение логарифмической трансформации к целевой переменной нормализует распределение

In [17]:
# Улучшение модели классификации
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline

print("Подбор гиперпараметров для задачи классификации")
print("=" * 50)

# Создание pipeline для предобработки и обучения
classification_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', KNeighborsClassifier())
])

# Определение сетки гиперпараметров
param_grid_classification = {
    'classifier__n_neighbors': [3, 5, 7, 9, 11, 15],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski'],
    'classifier__p': [1, 2]
}

# Настройка кросс-валидации с сохранением соотношения классов
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Выполнение поиска оптимальных гиперпараметров
# Используется метрика recall как приоритетная для задач обнаружения
grid_search_clf = GridSearchCV(
    estimator=classification_pipeline,
    param_grid=param_grid_classification,
    cv=cv_strategy,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

print("Выполнение grid search для классификации...")
grid_search_clf.fit(clf_X_train, clf_y_train)

print(f"Оптимальные параметры: {grid_search_clf.best_params_}")
print(f"Лучший результат кросс-валидации (recall): {grid_search_clf.best_score_:.4f}")

# Сохранение лучшей модели
best_classifier = grid_search_clf.best_estimator_

Подбор гиперпараметров для задачи классификации
Выполнение grid search для классификации...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Оптимальные параметры: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 3, 'classifier__p': 1, 'classifier__weights': 'uniform'}
Лучший результат кросс-валидации (recall): 0.9998
Оптимальные параметры: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 3, 'classifier__p': 1, 'classifier__weights': 'uniform'}
Лучший результат кросс-валидации (recall): 0.9998


In [18]:
# Улучшение модели регрессии с feature engineering
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import re

print("Feature engineering для задачи регрессии")
print("=" * 50)

# Загрузка исходных данных
regression_data = pd.read_csv("datasets/used_cars.csv")
data_processed = regression_data.copy()

# Предобработка признака пробега
print("Обработка признака пробега...")
def clean_mileage(mileage_str):
    """Извлечение числового значения пробега из строки"""
    if pd.isna(mileage_str):
        return np.nan
    # Извлечение числа из строки вида "51,000 mi."
    cleaned = re.sub(r'[^\d]', '', str(mileage_str))
    return float(cleaned) if cleaned else np.nan

data_processed['mileage_numeric'] = data_processed['milage'].apply(clean_mileage)

# Создание признака возраста автомобиля
current_year = 2023
data_processed['vehicle_age'] = current_year - data_processed['model_year']

# Обработка целевой переменной (цены)
prices_cleaned = (
    data_processed["price"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
    .astype(float)
)

# Удаление экстремальных выбросов в ценах (выше 99.5 перцентиля)
price_threshold = prices_cleaned.quantile(0.995)
mask = prices_cleaned <= price_threshold
data_processed = data_processed[mask]
prices_cleaned = prices_cleaned[mask]

print(f"Удалено {(~mask).sum()} экстремальных выбросов в ценах")
print(f"Новый диапазон цен: ${prices_cleaned.min():.0f} - ${prices_cleaned.max():.0f}")

# Обработка пропусков в категориальных признаках
categorical_columns = ['fuel_type', 'accident']
for col in categorical_columns:
    data_processed[col] = data_processed[col].fillna('Unknown')

# Удаление бесполезных признаков
columns_to_drop = ['price', 'clean_title']  # clean_title содержит только одно значение
data_processed = data_processed.drop(columns=columns_to_drop, errors='ignore')

# Создание dummy-переменных для категориальных признаков
features_encoded = pd.get_dummies(data_processed, drop_first=True)

# Группировка редких категорий (если категория встречается менее 10 раз)
for col in features_encoded.columns:
    if features_encoded[col].dtype == 'uint8':  # dummy переменные
        if features_encoded[col].sum() < 10:
            features_encoded = features_encoded.drop(columns=[col])

print(f"Количество признаков после feature engineering: {features_encoded.shape[1]}")

# Разделение на признаки и целевую переменную
X_regression = features_encoded
y_regression = prices_cleaned

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

print("\nПодбор гиперпараметров для регрессии")
print("=" * 40)

# Создание pipeline для регрессии
regression_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('regressor', KNeighborsRegressor())
])

# Определение сетки гиперпараметров для регрессии
param_grid_regression = {
    'regressor__n_neighbors': [3, 5, 7, 9, 11, 15, 20],
    'regressor__weights': ['uniform', 'distance'],
    'regressor__metric': ['euclidean', 'manhattan'],
    'regressor__p': [1, 2]
}

# Настройка кросс-валидации
cv_regression = KFold(n_splits=5, shuffle=True, random_state=42)

# Выполнение поиска оптимальных гиперпараметров
grid_search_reg = GridSearchCV(
    estimator=regression_pipeline,
    param_grid=param_grid_regression,
    cv=cv_regression,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("Выполнение grid search для регрессии...")
grid_search_reg.fit(X_train_reg, y_train_reg)

print(f"Оптимальные параметры: {grid_search_reg.best_params_}")
print(f"Лучший результат кросс-валидации (RMSE): {-grid_search_reg.best_score_:.2f}")

# Сохранение лучшей модели
best_regressor = grid_search_reg.best_estimator_

Feature engineering для задачи регрессии
Обработка признака пробега...
Удалено 21 экстремальных выбросов в ценах
Новый диапазон цен: $2000 - $359991
Количество признаков после feature engineering: 6404

Подбор гиперпараметров для регрессии
Выполнение grid search для регрессии...
Fitting 5 folds for each of 56 candidates, totalling 280 fits
Количество признаков после feature engineering: 6404

Подбор гиперпараметров для регрессии
Выполнение grid search для регрессии...
Fitting 5 folds for each of 56 candidates, totalling 280 fits
Оптимальные параметры: {'regressor__metric': 'manhattan', 'regressor__n_neighbors': 3, 'regressor__p': 1, 'regressor__weights': 'distance'}
Лучший результат кросс-валидации (RMSE): 29637.65
Оптимальные параметры: {'regressor__metric': 'manhattan', 'regressor__n_neighbors': 3, 'regressor__p': 1, 'regressor__weights': 'distance'}
Лучший результат кросс-валидации (RMSE): 29637.65


In [19]:
# Сравнение результатов базовой и улучшенной моделей

print("Сравнение качества моделей классификации")
print("=" * 50)

# Получение предсказаний улучшенной модели классификации
y_pred_enhanced = best_classifier.predict(clf_X_test)
y_proba_enhanced = best_classifier.predict_proba(clf_X_test)[:, 1]

# Вычисление метрик для улучшенной модели
metrics_enhanced = {
    'accuracy': accuracy_score(clf_y_test, y_pred_enhanced),
    'precision': precision_score(clf_y_test, y_pred_enhanced),
    'recall': recall_score(clf_y_test, y_pred_enhanced),
    'f1': f1_score(clf_y_test, y_pred_enhanced),
    'roc_auc': roc_auc_score(clf_y_test, y_proba_enhanced)
}

# Создание сравнительной таблицы
metrics_baseline = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'roc_auc': roc_auc
}

classification_comparison = pd.DataFrame({
    'Baseline': list(metrics_baseline.values()),
    'Enhanced': list(metrics_enhanced.values())
}, index=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'])

print("Результаты сравнения метрик:")
print(classification_comparison.round(4))

improvement_clf = classification_comparison['Enhanced'] - classification_comparison['Baseline']
print(f"\nИзменение метрик (Enhanced - Baseline):")
print(improvement_clf.round(4))

print("\n" + "=" * 50)
print("Сравнение качества моделей регрессии")
print("=" * 50)

# Получение предсказаний улучшенной модели регрессии
y_pred_reg_enhanced = best_regressor.predict(X_test_reg)

# Вычисление метрик для улучшенной модели регрессии
mae_enhanced = mean_absolute_error(y_test_reg, y_pred_reg_enhanced)
mse_enhanced = mean_squared_error(y_test_reg, y_pred_reg_enhanced)
rmse_enhanced = np.sqrt(mse_enhanced)
r2_enhanced = r2_score(y_test_reg, y_pred_reg_enhanced)

# Создание сравнительной таблицы для регрессии
regression_comparison = pd.DataFrame({
    'Baseline': [mae, rmse, r2],
    'Enhanced': [mae_enhanced, rmse_enhanced, r2_enhanced]
}, index=['MAE', 'RMSE', 'R²'])

print("Результаты сравнения метрик:")
print(regression_comparison.round(4))

improvement_reg = regression_comparison['Enhanced'] - regression_comparison['Baseline']
print(f"\nИзменение метрик (Enhanced - Baseline):")
print(improvement_reg.round(4))

# Расчет процентного улучшения
print(f"\nПроцентное изменение метрик:")
for metric in regression_comparison.index:
    baseline_val = regression_comparison.loc[metric, 'Baseline']
    if baseline_val != 0:
        percent_change = ((regression_comparison.loc[metric, 'Enhanced'] - baseline_val) / 
                         abs(baseline_val)) * 100
        print(f"{metric}: {percent_change:.1f}%")

Сравнение качества моделей классификации
Результаты сравнения метрик:
           Baseline  Enhanced
Accuracy     0.9992    0.9996
Precision    0.9991    0.9996
Recall       0.9998    0.9999
F1-Score     0.9994    0.9997
ROC-AUC      0.9997    0.9999

Изменение метрик (Enhanced - Baseline):
Accuracy     0.0004
Precision    0.0004
Recall       0.0001
F1-Score     0.0003
ROC-AUC      0.0001
dtype: float64

Сравнение качества моделей регрессии
Результаты сравнения метрик:
           Baseline  Enhanced
Accuracy     0.9992    0.9996
Precision    0.9991    0.9996
Recall       0.9998    0.9999
F1-Score     0.9994    0.9997
ROC-AUC      0.9997    0.9999

Изменение метрик (Enhanced - Baseline):
Accuracy     0.0004
Precision    0.0004
Recall       0.0001
F1-Score     0.0003
ROC-AUC      0.0001
dtype: float64

Сравнение качества моделей регрессии
Результаты сравнения метрик:
         Baseline    Enhanced
MAE    30653.8242  14709.7160
RMSE  142710.7012  30201.9161
R²         0.0036      0.4394

Изм

### Анализ результатов улучшения бейзлайна

**Результаты для задачи классификации (детекция дыма):**

Базовая модель изначально демонстрировала исключительно высокое качество (recall = 99.98%), что указывает на высокую сепарабельность классов в данных IoT-сенсоров. Оптимизация гиперпараметров обеспечила незначительные, но стабильные улучшения:

- Recall: 99.98% → 99.99% (+0.01%)
- Оптимальные параметры: Manhattan расстояние, k=3, uniform веса
- Все метрики превышают 99.96%

Малые улучшения объясняются тем, что данные уже обладают отличной структурой для алгоритма KNN. Основную ценность представляет стабилизация результатов и подтверждение оптимальности выбранных параметров.

**Результаты для задачи регрессии (цены автомобилей):**

Демонстрировано кардинальное улучшение качества модели благодаря комплексной предобработке данных:

**Достигнутые результаты:**
- R²: 0.004 → 0.44 (+12,165% улучшение)
- RMSE: $142,711 → $30,202 (-78.8% снижение ошибки)
- MAE: $30,654 → $14,710 (-52.0% снижение ошибки)

**Ключевые факторы успеха:**
1. **Очистка выбросов**: удаление 21 экстремального значения цен (выше $360k)
2. **Корректная обработка пробега**: преобразование текстового формата "51,000 mi." в числовой
3. **Feature engineering**: создание 6,404 признаков через dummy-кодирование и группировку редких категорий
4. **Оптимальные гиперпараметры**: Manhattan расстояние, k=3, distance веса

**Общие выводы:**

1. **Критичность предобработки для KNN**: Алгоритм показывает кардинально разные результаты в зависимости от качества входных данных. Для числовых данных высокого качества (сенсоры IoT) обеспечивает отличную performance, для "грязных" табличных данных требует тщательной очистки.

2. **Эффективность Manhattan расстояния**: Для обеих задач Manhattan метрика оказалась оптимальной, что может указывать на наличие категориальных или дискретных признаков в данных.

3. **Важность малого k**: Оптимальное значение k=3 для обеих задач подтверждает эффективность локального подхода при наличии качественных признаков.

4. **ROI предобработки**: Инвестиции времени в feature engineering для регрессии обеспечили рост объяснительной способности модели с практически нулевой до приемлемой (44% дисперсии).

## Собственная реализация алгоритма KNN

В данном разделе представлена самостоятельная имплементация алгоритма k-ближайших соседей для задач классификации и регрессии с последующим сравнением с библиотечными реализациями.

In [20]:
# Простая реализация KNN
import numpy as np
from collections import Counter

class SimpleKNNClassifier:
    """Простая реализация KNN для классификации"""
    
    def __init__(self, k=5):
        self.k = k
    
    def fit(self, X, y):
        """Сохраняем обучающие данные"""
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self
    
    def euclidean_distance(self, point1, point2):
        """Вычисление евклидова расстояния между двумя точками"""
        return np.sqrt(np.sum((point1 - point2) ** 2))
    
    def predict_one(self, x):
        """Предсказание для одной точки"""
        # Вычисляем расстояния до всех обучающих точек
        distances = []
        for i, x_train in enumerate(self.X_train):
            dist = self.euclidean_distance(x, x_train)
            distances.append((dist, self.y_train[i]))
        
        # Сортируем по расстоянию и берем k ближайших
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:self.k]
        
        # Голосование - выбираем самый частый класс
        k_nearest_labels = [label for _, label in k_nearest]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]
    
    def predict(self, X):
        """Предсказание для массива точек"""
        predictions = []
        for x in X:
            pred = self.predict_one(x)
            predictions.append(pred)
        return np.array(predictions)

class SimpleKNNRegressor:
    """Простая реализация KNN для регрессии"""
    
    def __init__(self, k=5):
        self.k = k
    
    def fit(self, X, y):
        """Сохраняем обучающие данные"""
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self
    
    def euclidean_distance(self, point1, point2):
        """Вычисление евклидова расстояния между двумя точками"""
        return np.sqrt(np.sum((point1 - point2) ** 2))
    
    def predict_one(self, x):
        """Предсказание для одной точки"""
        # Вычисляем расстояния до всех обучающих точек
        distances = []
        for i, x_train in enumerate(self.X_train):
            dist = self.euclidean_distance(x, x_train)
            distances.append((dist, self.y_train[i]))
        
        # Сортируем по расстоянию и берем k ближайших
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:self.k]
        
        # Берем среднее значение k ближайших соседей
        k_nearest_values = [value for _, value in k_nearest]
        return np.mean(k_nearest_values)
    
    def predict(self, X):
        """Предсказание для массива точек"""
        predictions = []
        for x in X:
            pred = self.predict_one(x)
            predictions.append(pred)
        return np.array(predictions)

print("Простая реализация KNN создана!")

Простая реализация KNN создана!


In [21]:
# Тестирование простой реализации KNN

print("Тестирование простой реализации KNN")
print("=" * 50)

# Тестирование на маленьком подмножестве данных для быстрой проверки
print("\nИспользуем подмножество данных для тестирования...")

# Берем первые 1000 образцов для классификации (ускорение)
clf_X_small = clf_X_train_scaled[:1000]
clf_y_small = clf_y_train[:1000]
clf_X_test_small = clf_X_test_scaled[:100]  
clf_y_test_small = clf_y_test[:100]

# Берем первые 500 образцов для регрессии
reg_X_small = reg_X_train_scaled[:500]
reg_y_small = reg_y_train[:500]
reg_X_test_small = reg_X_test_scaled[:50]
reg_y_test_small = reg_y_test[:50]

print("Размеры тестовых данных:")
print(f"Классификация: train={clf_X_small.shape}, test={clf_X_test_small.shape}")
print(f"Регрессия: train={reg_X_small.shape}, test={reg_X_test_small.shape}")

Тестирование простой реализации KNN

Используем подмножество данных для тестирования...
Размеры тестовых данных:
Классификация: train=(1000, 13), test=(100, 13)
Регрессия: train=(500, 6457), test=(50, 6457)


In [28]:
# Тестирование классификации
print("\n1. Тестирование классификации")
print("-" * 30)

# Наша реализация
my_clf = SimpleKNNClassifier(k=5)
my_clf.fit(clf_X_small, clf_y_small)
my_pred = my_clf.predict(clf_X_test_small)

# Sklearn реализация
sklearn_clf = KNeighborsClassifier(n_neighbors=5)
sklearn_clf.fit(clf_X_small, clf_y_small)
sklearn_pred = sklearn_clf.predict(clf_X_test_small)

# Сравнение результатов
my_accuracy = accuracy_score(clf_y_test_small, my_pred)
sklearn_accuracy = accuracy_score(clf_y_test_small, sklearn_pred)

print(f"Моя реализация - Accuracy: {my_accuracy:.4f}")
print(f"Sklearn - Accuracy: {sklearn_accuracy:.4f}")
print(f"Разница: {abs(my_accuracy - sklearn_accuracy):.4f}")

# Тестирование регрессии
print("\n2. Тестирование регрессии")
print("-" * 30)

# Наша реализация
my_reg = SimpleKNNRegressor(k=5)
my_reg.fit(reg_X_small, reg_y_small)
my_reg_pred = my_reg.predict(reg_X_test_small)

# Sklearn реализация
sklearn_reg = KNeighborsRegressor(n_neighbors=5)
sklearn_reg.fit(reg_X_small, reg_y_small)
sklearn_reg_pred = sklearn_reg.predict(reg_X_test_small)

# Сравнение результатов
my_r2 = r2_score(reg_y_test_small, my_reg_pred)
sklearn_r2 = r2_score(reg_y_test_small, sklearn_reg_pred)

print(f"Моя реализация - R²: {my_r2:.4f}")
print(f"Sklearn - R²: {sklearn_r2:.4f}")
print(f"Разница: {abs(my_r2 - sklearn_r2):.4f}")

print("\n" + "=" * 50)
print("Выводы:")
if abs(my_accuracy - sklearn_accuracy) < 0.01 and abs(my_r2 - sklearn_r2) < 0.01:
    print(" Моя реализация работает корректно!")
else:
    print(" Есть различия, но это нормально для простой реализации")

print(f"Моя реализация дает результаты близкие к sklearn")


1. Тестирование классификации
------------------------------
Моя реализация - Accuracy: 1.0000
Sklearn - Accuracy: 1.0000
Разница: 0.0000

2. Тестирование регрессии
------------------------------
Моя реализация - Accuracy: 1.0000
Sklearn - Accuracy: 1.0000
Разница: 0.0000

2. Тестирование регрессии
------------------------------
Моя реализация - R²: 0.0820
Sklearn - R²: 0.0820
Разница: 0.0000

Выводы:
 Моя реализация работает корректно!
Моя реализация дает результаты близкие к sklearn
Моя реализация - R²: 0.0820
Sklearn - R²: 0.0820
Разница: 0.0000

Выводы:
 Моя реализация работает корректно!
Моя реализация дает результаты близкие к sklearn


In [27]:
# Сравнение с улучшенными моделями из пункта 3
print("\n3. Сравнение с улучшенными моделями")
print("-" * 40)

print("Напомним результаты из пункта 3:")
print("\nКлассификация:")
print("- Baseline sklearn: Recall = 99.98%")
print("- Улучшенная sklearn: Recall = 99.99%")
print(f"- Моя реализация: Accuracy = {my_accuracy:.2%}")

print("\nРегрессия:")
print("- Baseline sklearn: R² = 0.004")
print("- Улучшенная sklearn: R² = 0.44") 
print(f"- Наша простая реализация: R² = {my_r2:.3f}")

print("\nОбщие выводы:")
print("1. Для качественных данных (классификация) - моей реализация работает хорошо")
print("2. Для сложных данных (регрессия) - важна предобработка и оптимизация")
print("3. Sklearn имеет множество оптимизаций, которых нет в моей реализации")


3. Сравнение с улучшенными моделями
----------------------------------------
Напомним результаты из пункта 3:

Классификация:
- Baseline sklearn: Recall = 99.98%
- Улучшенная sklearn: Recall = 99.99%
- Моя реализация: Accuracy = 100.00%

Регрессия:
- Baseline sklearn: R² = 0.004
- Улучшенная sklearn: R² = 0.44
- Наша простая реализация: R² = 0.082

Общие выводы:
1. Для качественных данных (классификация) - моей реализация работает хорошо
2. Для сложных данных (регрессия) - важна предобработка и оптимизация
3. Sklearn имеет множество оптимизаций, которых нет в моей реализации


### Выводы по собственной реализации KNN

**Что удалось реализовать:**

Я написал простую версию алгоритма k-ближайших соседей с нуля:
- Для классификации: голосование большинства среди k соседей
- Для регрессии: усреднение значений k соседей  
- Расстояние между точками вычисляется по евклидовой метрике

**Как работает алгоритм:**

1. Сохраняем все обучающие данные (поэтому KNN называют "ленивым")
2. Для нового объекта считаем расстояние до всех обучающих точек
3. Выбираем k ближайших соседей
4. Принимаем решение:
   - Классификация: берем самый частый класс среди соседей
   - Регрессия: считаем среднее значение соседей

**Результаты тестирования:**

Моя реализация показала отличные результаты:
- Классификация: точность 100% (такая же как у sklearn)
- Регрессия: R² = 0.082 (полностью совпадает с sklearn)
- Разница с библиотечной реализацией: 0.0000

**Сравнение с sklearn:**

- Правильность работы: моя версия дает точно такие же результаты
- Скорость: sklearn работает намного быстрее благодаря оптимизациям
- Возможности: в sklearn есть разные метрики расстояния, веса, алгоритмы поиска

**Что я понял в процессе работы:**

1. KNN действительно один из самых простых алгоритмов машинного обучения
2. Основная идея интуитивна: похожие объекты должны иметь похожие свойства
3. Алгоритм очень чувствителен к масштабу признаков - поэтому нужна нормализация
4. На больших данных простая реализация будет очень медленной
5. Качество результата сильно зависит от предобработки данных

**Практическая ценность:**

Написание KNN с нуля помогло мне:
- Лучше понять принципы работы алгоритма
- Увидеть важность выбора количества соседей
- Осознать проблемы масштабируемости
- Оценить качество профессиональных библиотек

В итоге получилась рабочая реализация, которая корректно решает обе задачи и дает результаты не хуже стандартной библиотеки sklearn.